In [2]:
# Import required libraries
import pandas as pd
import numpy as np
from textblob import TextBlob
import time
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import warnings
import socket

warnings.filterwarnings('ignore')

# -------------------------------------------------------------------
# 1. Load the India news headlines dataset from your local file.
# Update the file path if needed.
try:
    data = pd.read_csv(r"C:\Users\hp\Downloads\archive (11)\india-news-headlines.csv")
except Exception as e:
    print("Error loading dataset:", e)
    data = pd.DataFrame()

# Determine which column to use as the headline.
# We assume the dataset has a column named "headline".
if 'headline' in data.columns:
    headline_col = 'headline'
else:
    # If not, list available columns and use the first column.
    print("Available columns in dataset:", data.columns.tolist())
    headline_col = data.columns[0]

# -------------------------------------------------------------------
# 2. Define a function to compute sentiment using TextBlob.
def get_sentiment(text):
    # Returns a sentiment polarity score between -1 (very negative) and 1 (very positive)
    return TextBlob(str(text)).sentiment.polarity

# -------------------------------------------------------------------
# 3. Define a function to simulate real-time tweet sentiment by randomly sampling headlines.
def simulate_realtime_tweets(n_tweets=5):
    print("Starting real-time news sentiment simulation...")
    sentiments = []
    timestamps = []
    
    for i in range(n_tweets):
        # Randomly sample a headline from the dataset.
        tweet = data[headline_col].sample(1).iloc[0]
        sentiment = get_sentiment(tweet)
        sentiments.append(sentiment)
        timestamps.append(time.strftime('%Y-%m-%d %H:%M:%S'))
        print(f"Tweet {i+1}: {tweet[:50]}... Sentiment: {sentiment:.2f}")
        time.sleep(1)
    
    return pd.DataFrame({'Timestamp': timestamps, 'Sentiment': sentiments})

# Initialize the stream with a few simulated tweet sentiments.
tweet_data = simulate_realtime_tweets(n_tweets=5)

# -------------------------------------------------------------------
# 4. Helper function to get a free port.
def get_free_port():
    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    s.bind(("", 0))  # Let OS assign a free port.
    port = s.getsockname()[1]
    s.close()
    return port

free_port = get_free_port()

# -------------------------------------------------------------------
# 5. Create the Dash app for the real-time sentiment dashboard.
app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("Real-Time India News Headlines Sentiment Analysis"),
    dcc.Graph(id='sentiment-graph'),
    dcc.Interval(id='interval-component', interval=5*1000, n_intervals=0)
])

@app.callback(
    Output('sentiment-graph', 'figure'),
    [Input('interval-component', 'n_intervals')]
)
def update_graph(n):
    global tweet_data
    # Simulate one new tweet (headline) sentiment.
    new_data = simulate_realtime_tweets(n_tweets=1)
    tweet_data = pd.concat([tweet_data, new_data], ignore_index=True)
    
    # Create the figure (line chart) showing sentiment scores vs. time.
    fig = {
        'data': [
            {
                'x': tweet_data['Timestamp'],
                'y': tweet_data['Sentiment'],
                'type': 'line',
                'name': 'Headline Sentiment'
            }
        ],
        'layout': {
            'title': 'Real-Time India News Headlines Sentiment Analysis',
            'xaxis': {'title': 'Time'},
            'yaxis': {'title': 'Sentiment Score (-1 to 1)'}
        }
    }
    return fig

# -------------------------------------------------------------------
# 6. Run the Dash app on a free port.
if __name__ == '__main__':
    print(f"Starting Dash server... Open http://127.0.0.1:{free_port} in your browser.")
    # In updated versions of dash, simply use app.run() instead of app.run_server()
    app.run(debug=True, port=free_port)


Available columns in dataset: ['publish_date', 'headline_category', 'headline_text']
Starting real-time news sentiment simulation...


IndexError: invalid index to scalar variable.